In [155]:
import sys
#import h5py
import numpy as np
import operator
import pandas
import pymorphy2
import math


# Определяем входные данные
Наши входные данные это новостной массив за 2014 год.

In [3]:
input_path  = './data/' 
input_file = input_path + 'news_src.txt' 

# N-граммы

Построим таблицу вероятности встречаемости n-gramm для нашей модели <br>
Оценим вероятность слов в фразе: <br>
 *Президент США Барак Обама решил сняться в телепередаче с Беар Гриллсом.*

In [41]:
estimated_phrase = ['президент', 'сша', 'барак', 'обама', 'решить', 'сняться', 'в', 'телепередача', 'с', 'беар', 'гриллсом']

Unigramm
====

In [5]:
#
# получить набор unigramm согласно заданным параметрам
# - считаем частоты слов в датасете
# 
#
def get_unigramm_set(input_file, max_row_count=1e+5):
    unigramm_set = {}
    with open(input_file) as ifile:
        # итерация по всем документам
        for i, line in enumerate(ifile):
            parts = line.strip().split('\t')
            url = parts[0] # первое поле урл
            # пропускаем пустые документы
            if len(parts) <=1 : continue
            # 
            for j, sentens in enumerate(parts[1:]):
                sent = sentens.strip().split()
                slen = len(sent)
                if slen == 0: continue
                ar = []
                for iw, w in enumerate(sent):
                    
                    if w not in unigramm_set:
                        unigramm_set[w] = 0
                    unigramm_set[w] = unigramm_set[w] + 1 

            sys.stderr.write('\r'+str(i) + " total unigram: " + str(len(unigramm_set)))
            if i == max_row_count:
                break
    return unigramm_set               

* Получаем список униграмм 

In [6]:
unigramm_set = get_unigramm_set(input_file)

100000 total unigram: 248208

In [53]:
total_unigramms = 0
for v in unigramm_set.values():
    total_unigramms = total_unigramms + v

* Считаем вероятность появления каждой униграммы

In [54]:
def calculate_unigramm_probability_for(unigramm_set, total, phrase):
    sorted_inigramms = sorted(unigramm_set.items(), key=operator.itemgetter(1), reverse=True)
    n_top_words = {}
    for word in phrase:
        n_top_words[word] = (-1, 0)
        for i, uni in enumerate(sorted_inigramms):
             if uni[0] in n_top_words:
                n_top_words[uni[0]] = (i, uni[1] / float(total) )
    return  n_top_words

In [55]:
unigram_probs = calculate_unigramm_probability_for(unigramm_set, total_unigramms, estimated_phrase )

In [56]:
def get_prob_panda_table(probs):
    top_20_values = [[], []]
    for i, k in enumerate(estimated_phrase):
        top_20_values[0].append(int(probs[k][0]))
        top_20_values[1].append(probs[k][1])
    ###        
    # Делаем таблицу pandas
    ###
    dt = estimated_phrase
    values = np.asarray(top_20_values)
    values = values.reshape(2,11)
    index = ["num", "prob"]

    return pandas.DataFrame(values, index=index, columns=dt)    

* Выводим распределение для каждой униграммы в тексте

In [57]:
get_prob_panda_table(unigram_probs)


,президент,сша,барак,обама,решить,сняться,в,телепередача,с,беар,гриллсом
num,76.000000,116.00000,3395.000000,1799.000000,328.000000,4657.000000,0.000000,15613.000000,3.000000,14581.000000,21467.000000
prob,0.001167,0.00084,0.000037,0.000081,0.000395,0.000024,0.043321,0.000004,0.011905,0.000004,0.000002


* Самые вероятные униграммы -  самые частотные

In [59]:
for i, k in enumerate(sorted(unigramm_set.items(), key=operator.itemgetter(1), reverse=True)):
    print  k[0]  + ' %0.4f'%(k[1] / float(total_unigramms))
    if i == 5: break 

в 0.0433
и 0.0245
на 0.0194
с 0.0119
по 0.0098
что 0.0094


Bigramm
====
Оценим ту же самую фразу с точки зрения биграмной модели

Получаем биграммы с их частотой <br>
bigramm_set = {{}}

In [72]:


def get_bigram_set(input_file, max_row_count=1e+5):
    bigramm_set = {}
    with open(input_file) as ifile:
        # итерация по всем документам
        for i, line in enumerate(ifile):
            parts = line.strip().split('\t')
            url = parts[0] # первое поле урл
            # пропускаем пустые документы
            if len(parts) <=1 : continue
            # 
            for j, sentens in enumerate(parts[1:]):
                sent = sentens.strip().split()
                slen = len(sent)
                if slen == 0: continue
                ar = []
                for iw, w in enumerate(sent):
                    if iw + 1 < slen:
                        #bigram = w + ' ' + sent[iw+1]
                        if w not in bigramm_set:
                            bigramm_set[w] = {}
                        if sent[iw+1] not in bigramm_set[w]:
                            bigramm_set[w][sent[iw+1]] = 0
                        bigramm_set[w][sent[iw+1]] = bigramm_set[w][sent[iw+1]] + 1 

            sys.stderr.write('\r'+str(i) + " total bigram: " + str(len(bigramm_set)))
            if i == max_row_count:
                break
    return bigramm_set    

In [73]:
bigramm_set = get_bigram_set(input_file)

100000 total bigram: 228964

In [75]:
total_bigramms = 0
for v1 in bigramm_set.keys():
    for v2 in bigramm_set[v1].values():
        total_bigramms = total_bigramms + v2

* Рассчитываем биграмную вероятность для нашей фразы

In [113]:
def calculate_bigramm_probability_for(bigramm_set, unigramm_set, phrase):
    #sorted_gramms = sorted(bigramm_set.items(), key=operator.itemgetter(1), reverse=True)
    n_top_words = {}
    plen = len(phrase)
    for iw in range(0, plen-1):
        word1 = phrase[iw]
        word2 = phrase[iw+1]
        bigramm = word1 + ' ' + word2
        n_top_words[bigramm] = (-1, 0)
        if word1 in bigramm_set:
            sorted_gramms = sorted(bigramm_set[word1].items(), key=operator.itemgetter(1), reverse=True)
            for i, uni in enumerate(sorted_gramms):
                 if word1 + ' ' + uni[0] in n_top_words:
                    n_top_words[bigramm] = (i, uni[1] / float(unigramm_set[word1]) )
    return  n_top_words

In [114]:
bigram_probs = calculate_bigramm_probability_for(bigramm_set, unigramm_set, estimated_phrase )

In [115]:
def get_prob_panda_tableII(probs):
    top_20_values = [[], []]
    phrase_dt = []
    plen = len(estimated_phrase)
    for iw in range(0, plen-1):
        bigramm = estimated_phrase[iw] + ' ' + estimated_phrase[iw+1]
        phrase_dt.append(bigramm)
        top_20_values[0].append(int(probs[bigramm][0]))
        top_20_values[1].append(probs[bigramm][1])
    ###        
    # Делаем таблицу pandas
    ###
    dt = phrase_dt
    values = np.asarray(top_20_values)
    values = values.reshape(2,10)
    index = ["num", "prob"]

    return pandas.DataFrame(values, index=index, columns=dt)    

* Выводим таблицу вероятностей

In [117]:
get_prob_panda_tableII(bigram_probs)

,президент сша,сша барак,барак обама,обама решить,решить сняться,сняться в,в телепередача,телепередача с,с беар,беар гриллсом
num,4.000000,2.000000,0.000000,66.000000,264.000000,0.000000,3735.000000,16.000000,2105.000000,1.000000
prob,0.034506,0.022638,0.931526,0.002606,0.000671,0.451685,0.000018,0.014085,0.000062,0.316456


* Можем вывести топ слов, которые предшествуют данной биграмме. Например, для биграммы 'презедент США', наиболее вероятная биграмма 'презедент россии'  

In [121]:
sorted_gramms = sorted(bigramm_set['сша'].items(), key=operator.itemgetter(1), reverse=True)
for  uni in sorted_gramms[0:5]:
     print 'сша  ' + uni[0] + ' ' + str(uni[1] /  float(unigramm_set['сша'] ))

сша  и 0.0713204691638
сша  в 0.0707529322739
сша  барак 0.0226384159415
сша  по 0.0218817000883
сша  на 0.0180981208223


**Качество модели улучшилось, например, мы точно знаем, что после слова барак вероятнее всего стоит слово обама. Но к слову сша плохо подходит барак. Если сделаем триграмную модель, то получим большую вероятность того, что за презедент сша должен следовать барак** 

## 3-gramm

Самостоятельно

## Сглаживание

Реализовать сглаживание Лапласа на 3-грамной модели самостоятельно

----
Марковские цепи
===
Оценим вероятность фразы “Президент США Барак Обама решил сняться в телепередаче с Беар Гриллсом.” через марковскую цепь. <br>
Начальное состояние  - нахождение слова в начале предложения: pi <br>
Слово - состояние марковской цепи: s <br>
Вероятности перехода между словами - наши вероятностные события: x.
<br>
<br>
**Рассчитаем вероятности событий**
<br>
pi - начальные вероятности 
<br>
state_transitions - вероятности перехода

**Сначала получим статистику по взаимной встречаемости слов**

In [131]:
def get_markov_transition_stats(input_file, max_row_count=1e+5):
    state_transitions = {}
    pi = {}
    count_first_pos = 0
    with open(input_file) as ifile:
        # итерация по всем документам
        for i, line in enumerate(ifile):
            parts = line.strip().split('\t')
            url = parts[0] # первое поле урл
            # пропускаем пустые документы
            if len(parts) <=1 : continue
            # 
            for j, sentens in enumerate(parts[1:]):
                sent = sentens.strip().split()
                slen = len(sent)
                if slen == 0: continue
                ar = []
                for iw, w in enumerate(sent):
                    if iw == 0:
                        count_first_pos = count_first_pos + 1
                        if w not in pi:
                            pi [w]  = 0
                        pi[w] = pi[w] + 1
                    if iw + 1 < slen:
                        #bigram = w + ' ' + sent[iw+1]
                        if w not in state_transitions:
                            state_transitions[w] = {}
                        if sent[iw+1] not in state_transitions[w]:
                            state_transitions[w][sent[iw+1]] = 0
                        state_transitions[w][sent[iw+1]] = state_transitions[w][sent[iw+1]] + 1 

            sys.stderr.write('\r'+str(i) + " total bigram: " + str(len(state_transitions)))
            if i == max_row_count:
                break
    return state_transitions, pi, count_first_pos

In [132]:
state_transitions_stats, pi_stats, count_sent = get_markov_transition_stats(input_file)

100000 total bigram: 228964

**Теперь саму вероятность перехдов**

In [134]:
def calculate_transition_probability_for(state_transitions_stats, pi_stats, unigramm_set, count_sent):
    #sorted_gramms = sorted(bigramm_set.items(), key=operator.itemgetter(1), reverse=True)
    transition_probs = {}
    pi_probs = {}
    for word in pi_stats.keys():
        pi_probs[word] = pi_stats[word] / float(count_sent) 
        
    for word1 in state_transitions_stats.keys():
        if word1 not in transition_probs:
            transition_probs[word1] = {}
        for word2 in state_transitions_stats[word1]:
            if word2 not in transition_probs[word1]:
                transition_probs[word1][word2] = 0            
            transition_probs[word1][word2] = state_transitions_stats[word1][word2] / float(unigramm_set[word1])
    return  transition_probs, pi_probs

In [135]:
transition_probs, pi_probs = calculate_transition_probability_for(state_transitions_stats, pi_stats, unigramm_set, count_sent)

**Выводим вероятности перехода в виде таблички и считаем полную вероятность фразы**
Используем логарифм для того, что бы от умножения перейти к сложению

In [173]:
def get_trans_prob_panda_table(state_probs, pi, estimated_phrase):
    top_20_values = []
    phrase_dt = []
    plen = len(estimated_phrase)
    top_20_values.append(pi[estimated_phrase[0]])
    phrase_dt.append('pi (' + estimated_phrase[0] + ') ')
    
   
    for iw in range(0, plen-1):
        bigramm = 'p_s(' + estimated_phrase[iw] + '->' + estimated_phrase[iw+1] + ')'
        phrase_dt.append(bigramm)
        # есть вероятность
        if estimated_phrase[iw] in state_probs and estimated_phrase[iw+1]  in state_probs[estimated_phrase[iw]]:
            top_20_values.append(state_probs[estimated_phrase[iw]][estimated_phrase[iw+1]])
        else :
            top_20_values.append(1/float(len(state_probs)))
    
    result_prob = 1.
    log_prob = 0
    for prob in top_20_values:
        result_prob = result_prob * prob
        log_prob    = log_prob + math.log(prob) 
    
    top_20_values.append(result_prob)
    top_20_values.append(log_prob)
    phrase_dt.append('p(X1, ..., Xn)')
    phrase_dt.append('log(p(X1, ..., Xn))')
    ###        
    # Делаем таблицу pandas
    ###
    dt = phrase_dt
    values = np.asarray(top_20_values)
    values = values.reshape(1,13)
    index = ["probs"]

    return pandas.DataFrame(values, index=index, columns=dt)   

In [174]:
get_trans_prob_panda_table(transition_probs, pi_probs, estimated_phrase)

,pi (президент),p_s(президент->сша),p_s(сша->барак),p_s(барак->обама),p_s(обама->решить),p_s(решить->сняться),p_s(сняться->в),p_s(в->телепередача),p_s(телепередача->с),p_s(с->беар),p_s(беар->гриллсом),"p(X1, ..., Xn)","log(p(X1, ..., Xn))"
probs,0.002227,0.034506,0.022638,0.931526,0.002606,0.000671,0.451685,0.000018,0.014085,0.000062,0.316456,6.518518e-24,-53.387395


**Теперь посчитаем вероятность другой фразы заменив в базовой фразе одно слово**

In [177]:
false_estimated_phrase = ['президент', 'китая', 'барак', 'обама', 'решить', 'сняться', 'в', 'телепередача', 'с', 'беар', 'гриллсом']

In [178]:
get_trans_prob_panda_table(transition_probs, pi_probs, false_estimated_phrase)

,pi (президент),p_s(президент->китая),p_s(китая->барак),p_s(барак->обама),p_s(обама->решить),p_s(решить->сняться),p_s(сняться->в),p_s(в->телепередача),p_s(телепередача->с),p_s(с->беар),p_s(беар->гриллсом),"p(X1, ..., Xn)","log(p(X1, ..., Xn))"
probs,0.002227,0.000004,0.000004,0.931526,0.002606,0.000671,0.451685,0.000018,0.014085,0.000062,0.316456,1.591735e-31,-70.915313


***Теперь мы можем сравнивать вероятности для двух фраз и таким образом оценивать естесвенность данной языковой модели. Например можно оценить авторство, какому источнику принадлежит данный текст. Классификацию источников можно сделать самостоятельно*** 

Скрытые Марковские Модели
====

### Три задачи, которые нужно решить
1. Определить насколько наши наблюдения соответсвуют модели.
2. Выстроить последовательность согласно нашим наблюдениям.
3. Подобрать параметры модели.
